In [ ]:

import geopandas as gpd
import os
import warnings
import ast

warnings.filterwarnings("ignore")

# Base directory path
base_dir = r"C:\Users\natda\OneDrive - Northeastern University\Desktop\NatDave\Academics\PhD_NU\RESEARCH\Traffic_Stress\Boston"

# Construct file paths
roads_path = os.path.join(base_dir, "street_network.shp")
crossings_path = os.path.join(base_dir, "crossings.shp")
freeway_ramps_path = os.path.join(base_dir, "osm_freeway_ramps.gdb")

# Load shapefiles
roads = gpd.read_file(roads_path)
crossings = gpd.read_file(crossings_path)
freeway_ramps = gpd.read_file(freeway_ramps_path)

# Convert CRS_LEG from string to list
crossings['CRS_LEG'] = crossings['CRS_LEG'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)

# Create a new column FW_RAMP and initialize all values to None
crossings['FW_RAMP'] = None

# Reproject freeway ramps to match roads CRS
freeway_ramps = freeway_ramps.to_crs(roads.crs)

# Buffer freeway ramps by 2.5 meters
ramp_buffer = freeway_ramps.copy()
ramp_buffer['geometry'] = ramp_buffer.geometry.buffer(2.5)

# Find road segments that intersect either the ramp itself or the 2.5m buffer
ramp_intersecting_roads = roads[roads.geometry.intersects(ramp_buffer.unary_union)]
ramp_road_ids = set(ramp_intersecting_roads['unique_id'])

# Assign FW_RAMP = 1 where CRS_LEG contains an intersecting road segment
crossings['FW_RAMP'] = crossings['CRS_LEG'].apply(lambda x: 1 if any(seg in ramp_road_ids for seg in x) else None)

# Save the updated crossings shapefile
crossings.to_file(crossings_path, driver='ESRI Shapefile')

print("Updated crossings.shp with FW_RAMP attribute for freeway ramps (including 2.5m buffer).")

In [ ]:
crossings['FW_RAMP'].value_counts()